---
title: "Probabilistic Programming in Turing.jl"
author: "Mattias Villani"
editor: visual
---

#### IID normal model

$$
X_1,\ldots,X_n \vert \mu, \sigma^2 \overset{iid}{\sim} N(\mu, \sigma^2)
$$

Prior

$$
\sigma^2 \sim \chi^2(\nu_0, \sigma_0^2)
$$

$$
\mu \vert \sigma^2 \sim N\Big(\mu_0, \frac{\sigma^2}{\kappa_0}\Big)
$$

In [ ]:
using Turing, Plots, LaTeXStrings

ScaledInverseChiSq(ν,τ²) = InverseGamma(ν/2,ν*τ²/2) # Scaled Inv-χ² distribution

# Setting up the Turing model:
@model function iidnormal(x, μ₀, κ₀, ν₀, σ²₀)
    σ² ~ ScaledInverseChiSq(ν₀, σ²₀)
    θ ~ Normal(μ₀, √(σ²/κ₀))  # prior
    n = length(x)  # number of observations
    for i in 1:n
        x[i] ~ Normal(θ, √σ²) # model
    end
end

# Set up the observed data
x = [15.77,20.5,8.26,14.37,21.09]

# Set up the prior
μ₀ = 20; κ₀ = 1; ν₀ = 5; σ²₀ = 5^2

# Settings of the Hamiltonian Monte Carlo (HMC) sampler.
α = 0.8
postdraws = sample(iidnormal(x, μ₀, κ₀, ν₀, σ²₀), NUTS(α), 10000, discard_initial = 1000)

p1 = histogram(postdraws.value[:,2], yaxis = false, title = L"\mu")
p2 = histogram(sqrt.(postdraws.value[:,1]), yaxis = false, title = L"\sigma")
plot(p1, p2, layout = (1,2), size = (600,300))

#### **Poisson regression**

$$
y_i \vert \boldsymbol{x}_i \sim\mathrm{Poisson}(\lambda_i) \\
$$

with log link

$$
\lambda_{i} =\exp(\boldsymbol{x}_{i}^{\top}\boldsymbol{\beta})
$$

and multivariate normal prior

$$
\boldsymbol{\beta} \sim N(\boldsymbol{0}, \tau^2 \boldsymbol{I})
$$

![](figs/ebaydatatable.png)

In [ ]:
using Turing, CSV, Downloads, DataFrames, LinearAlgebra, LaTeXStrings, Plots

# Reading and transforming the eBay data
url = "https://github.com/mattiasvillani/BayesianLearningBook/raw/main/data/ebaybids/ebaybids.csv" 
df = CSV.read(Downloads.download(url), DataFrame)
n = size(df,1)
y = df[:,:NBidders]
X = [ones(n,1) log.(df.BookVal) .- mean(log.(df.BookVal)) df.ReservePriceFrac .- mean(df.ReservePriceFrac) df.MinorBlem df.MajorBlem  df.NegFeedback df.PowerSeller df.IDSeller df.Sealed]

varnames = ["intercept", "logbook", "startprice", "minblemish", "majblemish",    
      "negfeedback", "powerseller", "verified", "sealed"]

The Poisson regression with its prior is set up as Turing.jl using the @model macro:

In [ ]:
# Setting up the poisson regression model
@model function poissonReg(y, X, τ)
    p = size(X,2)
    β ~ filldist(Normal(0, τ), p)  # all βⱼ are iid Normal(0, τ)
    λ = exp.(X*β)
    n = length(y)  
    for i in 1:n
        y[i] ~ Poisson(λ[i]) 
    end
end

In [ ]:
# HMC sampling from posterior
p = size(X, 2)
μ = zeros(p)    # Prior mean
τ = 10          # Prior standard deviation Σ = τ²I
α = 0.70        # target acceptance probability in NUTS sampler
model = poissonReg(y, X, τ)
chain = sample(model, NUTS(α), 10000, discard_initial = 1000, verbose = false)
rateratio = exp.(chain.value) # exp(β) is the incidence rate ratio

gr(grid = false)
h = []
for i = 1:p
    ptmp = histogram(rateratio[:,i], nbins = 50, fillcolor = :steelblue, linecolor = nothing, 
        normalize = true, title = varnames[i], xlab = L"\exp(\beta_{%$(i-1)})", 
        yaxis = false, fillopacity = 0.5, label = "")    
    push!(h, ptmp)
end
plot(h..., size = (600,600), legend = :right)

Extensions to more complex models is easy. Negative binomial regression

$$
y_{i}\vert\boldsymbol{x}_{i} \sim\mathrm{NegBinomial}\left(\psi,p=\frac{\psi}{\psi+\lambda_{i}}\right),\quad\lambda_{i}=\exp(\boldsymbol{x}_{i}^{\top}\boldsymbol{\beta})
$$

``` julia
# Negative binomial regression 
@model function negbinomialReg(y, X, τ, μ₀, σ₀)
    p = size(X,2)
    β ~ filldist(Normal(0, τ), p)  # all βⱼ are iid Normal(0, τ)
    λ = exp.(X*β)
    ψ ~ LogNormal(μ₀, σ₀)             # log of overdispersion parameter
    n = length(y)  
    for i in 1:n
        y[i] ~ NegativeBinomial(ψ, ψ/(ψ + λ[i])) # mean is λ here, but var = λ(1 + λ/ψ) 
    end
end

μ₀ = 0   # Prior mean of log(ψ), where ψ is the overdispersion parameter
σ₀ = 10  
α = 0.70  # target acceptance probability in NUTS sampler
model = negbinomialReg(y, X, τ, μ₀, σ₀)
chain = sample(model, NUTS(α), 10000, discard_initial = 1000)
rateratio = exp.(chain.value)
```

![](figs/ebay_negbin_overdispersion.png){width="500"}

#### Variational inference in Turing.jl

``` julia
# Variational inference assuming posterior is independent normals
nSamples = 10
nGradSteps = 1000
approx_post = vi(model, ADVI(nSamples, nGradSteps)) # error when I try it now.
approx_post.dist.m # mean of variational approximation
approx_post.dist.σ # stdev of variational approximation
βsample = rand(approx_post, 10000)
```

![](figs/ebay_post_meanratio_vs_advi.png){width="800"}